In [4]:
import os
import xml.etree.ElementTree as ET
from sionna.rt import load_scene, SceneObject, RadioMaterial, PlanarArray, Transmitter
import numpy as np
import drjit as dr
from utils import parse_mitsuba_xml, build_sionna_scene

# ==============================================================================
# Main Execution
# ==============================================================================
xml_path = "dataset/scene_0/mitsuba_export/scene.xml"

if os.path.exists(xml_path):
    ids_list, files_dict, mats_dict, mats_set = parse_mitsuba_xml(xml_path)
    
    try:
        scene = build_sionna_scene(ids_list, files_dict, mats_dict, mats_set)
        
        # Optional: Test frequency update
        # Setting the frequency will trigger the callbacks and update material properties
        print("\n--- Testing Frequency Update (e.g., 2.4 GHz) ---")
        scene.frequency = 2.4e9 
        
        # Print updated properties for verification
        for name, mat in scene.radio_materials.items():
            # Note: Values might be tensors, print might show shapes/values
            print(f"Mat: {name:20} | Perm: {mat.relative_permittivity} | Cond: {mat.conductivity}")
        
        print("\n--- Testing Frequency Update (e.g., 3.5 GHz) ---")
        scene.frequency = 3.5e9 
        
        # Print updated properties for verification
        for name, mat in scene.radio_materials.items():
            # Note: Values might be tensors, print might show shapes/values
            print(f"Mat: {name:20} | Perm: {mat.relative_permittivity} | Cond: {mat.conductivity}")
            
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
else:
    print(f"File {xml_path} not found.")


--- Testing Frequency Update (e.g., 2.4 GHz) ---
Mat: Trunk                | Perm: [1.99] | Cond: [0.0120118]
Mat: Terrain_Concrete     | Perm: [5.24] | Cond: [0.0916312]
Mat: Terrain_MediumDry    | Perm: [13.7426] | Cond: [0.145818]
Mat: Terrain_Water        | Perm: [78] | Cond: [2]
Mat: Vegetation           | Perm: [1.02] | Cond: [0.002]
Mat: Building             | Perm: [5.24] | Cond: [0.0916312]

--- Testing Frequency Update (e.g., 3.5 GHz) ---
Mat: Trunk                | Perm: [1.99] | Cond: [0.0179982]
Mat: Terrain_Concrete     | Perm: [5.24] | Cond: [0.123087]
Mat: Terrain_MediumDry    | Perm: [13.2338] | Cond: [0.269711]
Mat: Terrain_Water        | Perm: [78] | Cond: [2]
Mat: Vegetation           | Perm: [1.02] | Cond: [0.002]
Mat: Building             | Perm: [5.24] | Cond: [0.123087]


In [ ]:
scene.tx_array = PlanarArray(num_rows=16,
                            num_cols=8,
                            vertical_spacing=0.5,
                            horizontal_spacing=0.5,
                            pattern="tr38901",
                            polarization="V")
# Create transmitter
pitch_deg = 4.0
yaw = [0.0, 2.0/3.0 * np.pi, 4.0/3.0 * np.pi]
pitch = np.deg2rad(pitch_deg)
pitch = -float(pitch)
tx = Transmitter(name="tx1",
                position=[-125,-125,60],
                orientation=[0.0,pitch,0.0],
                display_radius=10)
scene.add(tx)
tx = Transmitter(name="tx2",
                position=[-125,-125,60],
                orientation=[float(yaw[1]),pitch,0.0],
                display_radius=10)
scene.add(tx)
tx = Transmitter(name="tx3",
                position=[-125,-125,60],
                orientation=[float(yaw[2]),pitch,0.0],
                display_radius=10)
scene.add(tx)